# Setup

## Setup - Imports

In [ ]:
# python base imports
import datetime
import json

# import six
import six

print( "packages imported at " + str( datetime.datetime.now() ) )

## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

In [ ]:
%run ../django_init.py

In [ ]:
# django imports
from django.contrib.auth.models import User
from django.db.models import Max
from django.db.models import Min

# sourcenet imports
from context_text.shared.context_text_base import ContextTextBase

# context_analysis imports
from context_analysis.network.network_person_info import NetworkPersonInfo

# sourcenet imports
from context_text.models import Article
from context_text.models import Article_Author
from context_text.models import Article_Data
from context_text.models import Article_Subject
from context_text.models import Newspaper
from context_text.models import Person

# article coding
from context_text.article_coding.article_coder import ArticleCoder
#from context_text.article_coding.article_coding import ArticleCoding
from context_text.article_coding.open_calais_v2.open_calais_v2_article_coder import OpenCalaisV2ArticleCoder
from context_text.article_coding.open_calais_v2.open_calais_v2_api_response import OpenCalaisV2ApiResponse

# import class that actually processes requests for outputting networks.
from context_text.export.network_output import NetworkOutput

# context_text shared
from context_text.shared.context_text_base import ContextTextBase

print( "django model packages imported at " + str( datetime.datetime.now() ) )

## Setup - shared variables

In [ ]:
# get ArticleCoding instance.
#article_coding = ArticleCoding()

# automated coding user
automated_coder = ArticleCoder.get_automated_coding_user()

# newspapers for Grand Rapids Press and Detroit News.
grand_rapids_press = Newspaper.objects.get( newsbank_code = "GRPB" )
detroit_news = Newspaper.objects.get( newsbank_code = "DTNB" )

# OpenCalais v2 coder type
ocv2_coder_type = OpenCalaisV2ArticleCoder.CONFIG_APPLICATION

# Article_Data Overview

In [ ]:
article_data_qs = None
article_data_count = None

# get all Article_Data.
article_data_qs = Article_Data.objects.all()

# how many we starting with?
article_data_count = article_data_qs.count()

print( "Starting with {} total Article_Data instances.".format( article_data_count ) )

## Detect single-name people within Article_Data

- Moved this work off to separate work notebook: [analysis-omit_single_names.ipynb](./omit_single_names.ipynb)

## Filter on coder

### Only automated coder

In [ ]:
# get automated coder user.
#automated_coder = ArticleCoder.get_automated_coding_user()

# filter to just Article_Data coded by this user.
article_data_qs = article_data_qs.filter( coder = automated_coder )

# how many now?
article_data_count = article_data_qs.count()

print( "{} Article_Data instances for coder {}.".format( article_data_count, automated_coder ) )

### Only OpenCalais_v2

In [ ]:
my_coder_type = ocv2_coder_type

# filter to just Article_Data with coder_type of "OpenCalais_REST_API_v2"
article_data_qs = article_data_qs.filter( coder_type = my_coder_type )

# how many now?
article_data_count = article_data_qs.count()

print( "{} Article_Data instances for coder_type {}.".format( article_data_count, my_coder_type ) )

## Filter on paper

### Only Detroit News

In [ ]:
# get automated coder user.
#automated_coder = ArticleCoder.get_automated_coding_user()
my_newspaper = detroit_news

# filter to just Article_Data coded by this user.
article_data_qs = article_data_qs.filter( article__newspaper = my_newspaper )

# how many now?
article_data_count = article_data_qs.count()

print( "{} Article_Data instances for newspaper {}.".format( article_data_count, my_newspaper ) )

### Only Grand Rapids Press

In [ ]:
# get automated coder user.
#automated_coder = ArticleCoder.get_automated_coding_user()
my_newspaper = grand_rapids_press

# filter to just Article_Data coded by this user.
article_data_qs = article_data_qs.filter( article__newspaper = my_newspaper )

# how many now?
article_data_count = article_data_qs.count()

print( "{} Article_Data instances for newspaper {}.".format( article_data_count, my_newspaper ) )

# Article_Data for GRP analysis

In [ ]:
# init
grp_article_data_qs = None
article_data_count = None
work_qs = None

In [ ]:
# get all Article_Data.
grp_article_data_qs = Article_Data.objects.all()

# how many we starting with?
article_data_count = grp_article_data_qs.count()

print( "Starting with {} total Article_Data instances.".format( article_data_count ) )

## GRP - Only OpenCalais v.2

In [ ]:
my_coder_type = ocv2_coder_type

# filter to just Article_Data with coder_type of "OpenCalais_REST_API_v2"
grp_article_data_qs = grp_article_data_qs.filter( coder = automated_coder )
grp_article_data_qs = grp_article_data_qs.filter( coder_type = my_coder_type )

# how many now?
article_data_count = grp_article_data_qs.count()

print( "{} Article_Data instances for coder_type {}.".format( article_data_count, my_coder_type ) )

## GRP - Only Grand Rapids Press

In [ ]:
# get automated coder user.
#automated_coder = ArticleCoder.get_automated_coding_user()
my_newspaper = grand_rapids_press

# filter to just Article_Data coded by this user.
grp_article_data_qs = grp_article_data_qs.filter( article__newspaper = my_newspaper )

# how many now?
article_data_count = grp_article_data_qs.count()

print( "{} Article_Data instances for newspaper {}.".format( article_data_count, my_newspaper ) )

## GRP - date ranges

Get min and max publication dates for articles within Article_Data QuerySet.

- Django aggregation function guide: [https://docs.djangoproject.com/en/4.0/topics/db/aggregation/](https://docs.djangoproject.com/en/4.0/topics/db/aggregation/)

In [ ]:
# minimum publication date
work_qs = grp_article_data_qs
aggregate_value_dict = work_qs.aggregate( Min( "article__pub_date" ) )  
min_pubdate = aggregate_value_dict[ "article__pub_date__min" ]
print( "Min pubdate: {min_pubdate} ( {agg_dict} )".format( min_pubdate = min_pubdate, agg_dict = aggregate_value_dict ) )

In [ ]:
# maximum publication date
work_qs = grp_article_data_qs
aggregate_value_dict = work_qs.aggregate( Max( "article__pub_date" ) )  
max_pubdate = aggregate_value_dict[ "article__pub_date__max" ]
print( "Max pubdate: {max_pubdate} ( {agg_dict} )".format( max_pubdate = max_pubdate, agg_dict = aggregate_value_dict ) )

## GRP - missing names in Article_Subject

### GRP - Article_Subject - name

In [ ]:
# declare variables
empty_name_list = None
ad_id_to_empty_name_map = None
article_data = None
name_qs = None
name_count = None

# init
empty_name_list = list()
ad_id_to_empty_name_map = dict()

# ==> name

# get all Article_Subject...
name_qs = Article_Subject.objects.all()
name_count = name_qs.count()
print( "Article_Subject count: {}".format( name_count ) ) 

# ...with empty name...
name_qs = name_qs.filter( name__isnull = True )
name_count = name_qs.count()
print( "Empty name count: {}".format( name_count ) ) 

# ...just those that are related to an automated coder...
name_qs = name_qs.filter( article_data__coder = automated_coder )
name_count = name_qs.count()
print( "Empty name by automated coder count: {}".format( name_count ) )

# ...just those that are type "OpenCalais_REST_API_v2"...
name_qs = name_qs.filter( article_data__coder_type = ocv2_coder_type )
name_count = name_qs.count()
print( "Empty name by OpenCalais v2: {}".format( name_count ) )

# ...only Grand Rapids Press...?
name_qs = name_qs.filter( article_data__article__newspaper = grand_rapids_press )
name_count = name_qs.count()
print( "Empty name only GRP articles: {}".format( name_count ) )

### GRP - Article_Subject - verbatim_name

In [ ]:
# declare variables
empty_name_list = None
ad_id_to_empty_name_map = None
article_data = None
name_qs = None
name_count = None

# init
empty_name_list = list()
ad_id_to_empty_name_map = dict()

# ==> name

# get all Article_Subject...
name_qs = Article_Subject.objects.all()
name_count = name_qs.count()
print( "Article_Subject count: {}".format( name_count ) ) 

# ...with empty verbatim_name...
name_qs = name_qs.filter( verbatim_name__isnull = True )
name_count = name_qs.count()
print( "Empty verbatim_name count: {}".format( name_count ) ) 

# ...just those that are related to an automated coder...
name_qs = name_qs.filter( article_data__coder = automated_coder )
name_count = name_qs.count()
print( "Empty verbatim_name by automated coder count: {}".format( name_count ) )

# ...just those that are type "OpenCalais_REST_API_v2"...
name_qs = name_qs.filter( article_data__coder_type = ocv2_coder_type )
name_count = name_qs.count()
print( "Empty verbatim_name by OpenCalais v2: {}".format( name_count ) )

# ...only Grand Rapids Press...?
name_qs = name_qs.filter( article_data__article__newspaper = grand_rapids_press )
name_count = name_qs.count()
print( "Empty verbatim_name only GRP articles: {}".format( name_count ) )

### GRP - Article_Subject - lookup_name

In [ ]:
# declare variables
empty_name_list = None
ad_id_to_empty_name_map = None
article_data = None
name_qs = None
name_count = None

# init
empty_name_list = list()
ad_id_to_empty_name_map = dict()

# ==> name

# get all Article_Subject...
name_qs = Article_Subject.objects.all()
name_count = name_qs.count()
print( "Article_Subject count: {}".format( name_count ) ) 

# ...with empty lookup_name...
name_qs = name_qs.filter( lookup_name__isnull = True )
name_count = name_qs.count()
print( "Empty lookup_name count: {}".format( name_count ) ) 

# ...just those that are related to an automated coder...
name_qs = name_qs.filter( article_data__coder = automated_coder )
name_count = name_qs.count()
print( "Empty lookup_name by automated coder count: {}".format( name_count ) )

# ...just those that are type "OpenCalais_REST_API_v2"...
name_qs = name_qs.filter( article_data__coder_type = ocv2_coder_type )
name_count = name_qs.count()
print( "Empty lookup_name by OpenCalais v2: {}".format( name_count ) )

# ...only Grand Rapids Press...?
name_qs = name_qs.filter( article_data__article__newspaper = grand_rapids_press )
name_count = name_qs.count()
print( "Empty lookup_name only GRP articles: {}".format( name_count ) )

### GRP - Article_Subject - all three

In [ ]:
# declare variables
empty_name_list = None
ad_id_to_empty_name_map = None
article_data = None
name_qs = None
name_count = None

# init
empty_name_list = list()
ad_id_to_empty_name_map = dict()

# ==> name

# get all Article_Subject...
name_qs = Article_Subject.objects.all()
name_count = name_qs.count()
print( "Article_Subject count: {}".format( name_count ) ) 

# ...with empty name...
name_qs = name_qs.filter( name__isnull = True )
name_qs = name_qs.filter( verbatim_name__isnull = True )
name_qs = name_qs.filter( lookup_name__isnull = True )
name_count = name_qs.count()
print( "Empty all names count: {}".format( name_count ) ) 

# ...just those that are related to an automated coder...
name_qs = name_qs.filter( article_data__coder = automated_coder )
name_count = name_qs.count()
print( "Empty all names by automated coder count: {}".format( name_count ) )

# ...just those that are type "OpenCalais_REST_API_v2"...
name_qs = name_qs.filter( article_data__coder_type = ocv2_coder_type )
name_count = name_qs.count()
print( "Empty all names by OpenCalais v2: {}".format( name_count ) )

# ...only Grand Rapids Press...?
name_qs = name_qs.filter( article_data__article__newspaper = grand_rapids_press )
name_count = name_qs.count()
print( "Empty all names only GRP articles: {}".format( name_count ) )

### GRP - Article_Subject - fix no-names

In [ ]:
# get IDs of the Article_Subjects with no name info.
status_message = None
no_name_counter = None
no_name_instance = None
no_name_id = None
no_name_id_list = None
article_data = None
article_data_id = None
no_name_string = None
do_update = None

# declare variables - try to fix?
open_calais_response_helper = None
my_article_data = None
my_article_data_id = None
my_article_data_notes_qs = None
article_data_note = None
my_open_calais_json = None
my_person = None
my_person_id = None
person_ids_qs = None
person_ids_count = None
person_id_instance = None
open_calais_uuid = None
oc_person_json = None
oc_person_uuid_to_json_map = None
oc_person_json_count = None
oc_person_name = None
json_value_list = None

# init
open_calais_response_helper = OpenCalaisV2ApiResponse()
do_update = False

# loop over Article_Subject instances
no_name_counter = 0
name_qs = name_qs.order_by( 'id' )
for no_name_instance in name_qs:
    
    # increment counter
    no_name_counter += 1
    
    # init
    oc_person_uuid_to_json_map = dict()
    id_counter = 0
    
    status_message = "\nProcessing {counter}: {no_name_instance}".format(
        counter = no_name_counter,
        no_name_instance = no_name_instance
    )
    print( status_message )

    #=========================================================================#
    # Article_Data, Article_Data_Notes, and JSON into OpenCalaisV2ApiResponse
    #=========================================================================#
    
    # get Article_Data
    my_article_data = no_name_instance.article_data
    my_article_data_id = my_article_data.id
    
    # get Article_Data_Notes
    my_article_data_notes_qs = my_article_data.article_data_notes_set.all()
    my_article_data_notes_qs = my_article_data_notes_qs.filter( source = OpenCalaisV2ArticleCoder.CONFIG_APPLICATION )
    my_article_data_note = my_article_data_notes_qs.get()
    
    # get JSON
    my_open_calais_json = my_article_data_note.content_json
    
    # load it into open_calais_response_helper
    open_calais_response_helper.set_json_response_object( my_open_calais_json )
    
    #=========================================================================#
    # Person and OpenCalais UUID.
    #=========================================================================#

    # get person.
    my_person = no_name_instance.person
    my_person_id = my_person.id
    
    # get person's external UUIDs.
    person_ids_qs = my_person.person_external_uuid_set.all()
    
    # try to get OpenCalais ID.
    person_ids_qs = person_ids_qs.filter( source__istartswith = "OpenCalais_REST_API" )
    person_ids_count = person_ids_qs.count()
    if ( person_ids_count == 1 ):
        
        # got one - could be v1, could be v2, doesn't matter. Get instance.
        person_id_instance = person_ids_qs.get()
        
        # get ID value.
        open_calais_uuid = person_id_instance.uuid
        print( "- ----> UUID: {}".format( person_id_instance ) )
        
        # try to get JSON for UUID
        oc_person_json = open_calais_response_helper.get_item_from_response( open_calais_uuid )

        # got anything?
        if ( oc_person_json is not None ):

            # yes - add to list.
            oc_person_uuid_to_json_map[ open_calais_uuid ] = oc_person_json

        #-- END check if got anything back --#
        
    else:
        
        # multiple matches. Oy.
        status_message = "- ----> multiple IDs where source starts with 'OpenCalais_REST_API' - which to use?"
        print( status_message )
        
        id_counter = 0
        for person_id_instance in person_ids_qs:
            
            id_counter += 1
            
            print( "- {} - {}".format( id_counter, person_id_instance ) )
            
            # get ID value.
            open_calais_uuid = person_id_instance.uuid
            print( "- ----> UUID: {}".format( open_calais_uuid ) )

            # try to get JSON for UUID
            oc_person_json = open_calais_response_helper.get_item_from_response( open_calais_uuid )
            
            # got anything?
            if ( oc_person_json is not None ):
                
                # yes - add to list.
                oc_person_uuid_to_json_map[ open_calais_uuid ] = oc_person_json
            
            #-- END check if got anything back --#
            
        #-- END loop over multiple matching uuids --#
        
    #-- END check if single matching OpenCalais API ID --#
    
    # got an OpenCalais person JSON object?
    oc_person_json_count = len( oc_person_uuid_to_json_map )
    if ( oc_person_json_count > 0 ):
        
        # how many things in list?
        if ( oc_person_json_count == 1 ):
            
            # just one. Use it to fix. Get JSON.
            json_value_list = oc_person_uuid_to_json_map.values()
            json_value_list = list( json_value_list )
            oc_person_json = json_value_list[ 0 ]
            
            # at any rate, if one present, retrieve "name" property.
            oc_person_name = oc_person_json.get( "name", None )
            print( "- ----> found name in JSON: {} ( uuid: {} )".format( oc_person_name, open_calais_uuid ) )

            # store in no_name_instance
            no_name_instance.name = oc_person_name
            no_name_instance.verbatim_name = oc_person_name
            no_name_instance.lookup_name = oc_person_name

            if ( do_update == True ):

                no_name_instance.save()

            #-- END check if updating. --#
            
        elif ( oc_person_json_count > 1 ):
            
            # more than one matching user... Should not happen, I think?
            status_message = "- !!!! ERROR - MULTIPLE UUID for a person found in JSON. Should not happen. Details: {}".format( oc_person_uuid_to_json_map )
            print( status_message )
            
        else:
            
            # neither 1 or > 1, but greater than 0. Error.
            status_message = "- !!!! ERROR - neither 1 or > 1, but greater than 0. Impossible. Details: {}".format( oc_person_uuid_to_json_map )
            print( status_message )
            
        #-- END check for unexpected outcome --#
                
    else:
        
        # no person found in JSON - big error, not sure what we'll do.
        status_message = "- !!!! ERROR: Person NOT FOUND in JSON ( Person: {my_person}; Article_Data: {article_data}.".format(
            my_person = my_person,
            article_data = my_article_data
        )
        print( status_message )
        
    #-- END check to see if found person in JSON. --#
    
    # get IDs.
    no_name_id = no_name_instance.id
    article_data = no_name_instance.article_data
    article_data_id = article_data.id
    
    # output links
    print( "- Links:" )
    print( "" )
    print( "    - Article_Subject {no_name_id} ( https://research.local/research/admin/context_text/article_subject/{no_name_id}/change/ )".format( no_name_id = no_name_id ) )
    print( "    - Article_Data {AD_id} ( https://research.local/research/admin/context_text/article_data/{AD_id}/change/ )".format( AD_id = article_data_id ) )
    print( "    - Person {my_person_id} ( https://research.local/research/admin/context_text/person/{my_person_id}/change/ )".format( my_person_id = my_person_id ) )
    
#-- END loop over names QuerySet --#


## GRP - missing names in Article_Author

### GRP - Article_Author - name

In [ ]:
# declare variables
empty_name_list = None
ad_id_to_empty_name_map = None
article_data = None
name_qs = None
name_count = None

# init
empty_name_list = list()
ad_id_to_empty_name_map = dict()

# ==> name

# get all Article_Author...
name_qs = Article_Author.objects.all()
name_count = name_qs.count()
print( "Article_Author count: {}".format( name_count ) ) 

# ...with empty name...
name_qs = name_qs.filter( name__isnull = True )
name_count = name_qs.count()
print( "Empty name count: {}".format( name_count ) ) 

# ...just those that are related to an automated coder...
name_qs = name_qs.filter( article_data__coder = automated_coder )
name_count = name_qs.count()
print( "Empty name by automated coder count: {}".format( name_count ) )

# ...just those that are type "OpenCalais_REST_API_v2"...
name_qs = name_qs.filter( article_data__coder_type = ocv2_coder_type )
name_count = name_qs.count()
print( "Empty name by OpenCalais v2: {}".format( name_count ) )

# ...only Grand Rapids Press...?
name_qs = name_qs.filter( article_data__article__newspaper = grand_rapids_press )
name_count = name_qs.count()
print( "Empty name only GRP articles: {}".format( name_count ) )

### GRP - Article_Author - verbatim_name

In [ ]:
# declare variables
empty_name_list = None
ad_id_to_empty_name_map = None
article_data = None
name_qs = None
name_count = None

# init
empty_name_list = list()
ad_id_to_empty_name_map = dict()

# ==> name

# get all Article_Author...
name_qs = Article_Author.objects.all()
name_count = name_qs.count()
print( "Article_Author count: {}".format( name_count ) ) 

# ...with empty verbatim_name...
name_qs = name_qs.filter( verbatim_name__isnull = True )
name_count = name_qs.count()
print( "Empty verbatim_name count: {}".format( name_count ) ) 

# ...just those that are related to an automated coder...
name_qs = name_qs.filter( article_data__coder = automated_coder )
name_count = name_qs.count()
print( "Empty verbatim_name by automated coder count: {}".format( name_count ) )

# ...just those that are type "OpenCalais_REST_API_v2"...
name_qs = name_qs.filter( article_data__coder_type = ocv2_coder_type )
name_count = name_qs.count()
print( "Empty verbatim_name by OpenCalais v2: {}".format( name_count ) )

# ...only Grand Rapids Press...?
name_qs = name_qs.filter( article_data__article__newspaper = grand_rapids_press )
name_count = name_qs.count()
print( "Empty verbatim_name only GRP articles: {}".format( name_count ) )

### GRP - Article_Author - lookup_name

In [ ]:
# declare variables
empty_name_list = None
ad_id_to_empty_name_map = None
article_data = None
name_qs = None
name_count = None

# init
empty_name_list = list()
ad_id_to_empty_name_map = dict()

# ==> name

# get all Article_Author...
name_qs = Article_Author.objects.all()
name_count = name_qs.count()
print( "Article_Author count: {}".format( name_count ) ) 

# ...with empty lookup_name...
name_qs = name_qs.filter( lookup_name__isnull = True )
name_count = name_qs.count()
print( "Empty lookup_name count: {}".format( name_count ) ) 

# ...just those that are related to an automated coder...
name_qs = name_qs.filter( article_data__coder = automated_coder )
name_count = name_qs.count()
print( "Empty lookup_name by automated coder count: {}".format( name_count ) )

# ...just those that are type "OpenCalais_REST_API_v2"...
name_qs = name_qs.filter( article_data__coder_type = ocv2_coder_type )
name_count = name_qs.count()
print( "Empty lookup_name by OpenCalais v2: {}".format( name_count ) )

# ...only Grand Rapids Press...?
name_qs = name_qs.filter( article_data__article__newspaper = grand_rapids_press )
name_count = name_qs.count()
print( "Empty lookup_name only GRP articles: {}".format( name_count ) )

### GRP - Article_Author - all three

In [ ]:
# declare variables
empty_name_list = None
ad_id_to_empty_name_map = None
article_data = None
name_qs = None
name_count = None

# init
empty_name_list = list()
ad_id_to_empty_name_map = dict()

# ==> name

# get all Article_Author...
name_qs = Article_Author.objects.all()
name_count = name_qs.count()
print( "Article_Author count: {}".format( name_count ) ) 

# ...with empty name...
name_qs = name_qs.filter( name__isnull = True )
name_qs = name_qs.filter( verbatim_name__isnull = True )
name_qs = name_qs.filter( lookup_name__isnull = True )
name_count = name_qs.count()
print( "Empty all names count: {}".format( name_count ) ) 

# ...just those that are related to an automated coder...
name_qs = name_qs.filter( article_data__coder = automated_coder )
name_count = name_qs.count()
print( "Empty all names by automated coder count: {}".format( name_count ) )

# ...just those that are type "OpenCalais_REST_API_v2"...
name_qs = name_qs.filter( article_data__coder_type = ocv2_coder_type )
name_count = name_qs.count()
print( "Empty all names by OpenCalais v2: {}".format( name_count ) )

# ...only Grand Rapids Press...?
name_qs = name_qs.filter( article_data__article__newspaper = grand_rapids_press )
name_count = name_qs.count()
print( "Empty all names only GRP articles: {}".format( name_count ) )

### GRP - Article_Author - fix no-names

In [ ]:
# get IDs of the Article_Subjects with no name info.
status_message = None
no_name_counter = None
no_name_instance = None
no_name_id = None
no_name_id_list = None
article_data = None
article_data_id = None
no_name_string = None
do_update = None

# declare variables - try to fix?
open_calais_response_helper = None
my_article_data = None
my_article_data_id = None
my_article = None
my_article_id = None
my_article_data_notes_qs = None
article_data_note = None
my_open_calais_json = None
my_person = None
my_person_id = None
person_ids_qs = None
person_ids_count = None
person_id_instance = None
open_calais_uuid = None
oc_person_json = None
oc_person_uuid_to_json_map = None
oc_person_json_count = None
oc_person_name = None
json_value_list = None

# init
open_calais_response_helper = OpenCalaisV2ApiResponse()
do_update = False

# loop over Article_Subject instances
no_name_counter = 0
name_qs = name_qs.order_by( 'id' )
for no_name_instance in name_qs:
    
    # increment counter
    no_name_counter += 1
    
    # init
    oc_person_uuid_to_json_map = dict()
    id_counter = 0
    
    status_message = "\nProcessing {counter}: {no_name_instance}".format(
        counter = no_name_counter,
        no_name_instance = no_name_instance
    )
    print( status_message )

    #=========================================================================#
    # Article_Data, Article_Data_Notes, and JSON into OpenCalaisV2ApiResponse
    #=========================================================================#
    
    # get Article_Data
    my_article_data = no_name_instance.article_data
    my_article_data_id = my_article_data.id
    
    # get Article
    my_article = my_article_data.article
    my_article_id = my_article.id
    
    # get Article_Data_Notes
    my_article_data_notes_qs = my_article_data.article_data_notes_set.all()
    my_article_data_notes_qs = my_article_data_notes_qs.filter( source = OpenCalaisV2ArticleCoder.CONFIG_APPLICATION )
    my_article_data_note = my_article_data_notes_qs.get()
    
    # get JSON
    my_open_calais_json = my_article_data_note.content_json
    
    # load it into open_calais_response_helper
    open_calais_response_helper.set_json_response_object( my_open_calais_json )
    
    #=========================================================================#
    # Person and OpenCalais UUID.
    #=========================================================================#

    # get person.
    my_person = no_name_instance.person
    my_person_id = my_person.id
    
    # get person's external UUIDs.
    person_ids_qs = my_person.person_external_uuid_set.all()
    
    # try to get OpenCalais ID.
    person_ids_qs = person_ids_qs.filter( source__istartswith = "OpenCalais_REST_API" )
    person_ids_count = person_ids_qs.count()
    if ( person_ids_count == 1 ):
        
        # got one - could be v1, could be v2, doesn't matter. Get instance.
        person_id_instance = person_ids_qs.get()
        
        # get ID value.
        open_calais_uuid = person_id_instance.uuid
        print( "- ----> UUID: {}".format( person_id_instance ) )
        
        # try to get JSON for UUID
        oc_person_json = open_calais_response_helper.get_item_from_response( open_calais_uuid )

        # got anything?
        if ( oc_person_json is not None ):

            # yes - add to list.
            oc_person_uuid_to_json_map[ open_calais_uuid ] = oc_person_json

        #-- END check if got anything back --#
        
    else:
        
        # multiple matches. Oy.
        status_message = "- ----> multiple IDs where source starts with 'OpenCalais_REST_API' - which to use?"
        print( status_message )
        
        id_counter = 0
        for person_id_instance in person_ids_qs:
            
            id_counter += 1
            
            print( "- {} - {}".format( id_counter, person_id_instance ) )
            
            # get ID value.
            open_calais_uuid = person_id_instance.uuid
            print( "- ----> UUID: {}".format( open_calais_uuid ) )

            # try to get JSON for UUID
            oc_person_json = open_calais_response_helper.get_item_from_response( open_calais_uuid )
            
            # got anything?
            if ( oc_person_json is not None ):
                
                # yes - add to list.
                oc_person_uuid_to_json_map[ open_calais_uuid ] = oc_person_json
            
            #-- END check if got anything back --#
            
        #-- END loop over multiple matching uuids --#
        
    #-- END check if single matching OpenCalais API ID --#
    
    # got an OpenCalais person JSON object?
    oc_person_json_count = len( oc_person_uuid_to_json_map )
    if ( oc_person_json_count > 0 ):
        
        # how many things in list?
        if ( oc_person_json_count == 1 ):
            
            # just one. Use it to fix. Get JSON.
            json_value_list = oc_person_uuid_to_json_map.values()
            json_value_list = list( json_value_list )
            oc_person_json = json_value_list[ 0 ]
            
            # at any rate, if one present, retrieve "name" property.
            oc_person_name = oc_person_json.get( "name", None )
            print( "- ----> found name in JSON: {} ( uuid: {} )".format( oc_person_name, open_calais_uuid ) )

            # store in no_name_instance
            no_name_instance.name = oc_person_name
            no_name_instance.verbatim_name = oc_person_name
            no_name_instance.lookup_name = oc_person_name

            if ( do_update == True ):

                no_name_instance.save()

            #-- END check if updating. --#
            
        elif ( oc_person_json_count > 1 ):
            
            # more than one matching user... Should not happen, I think?
            status_message = "- !!!! ERROR - MULTIPLE UUID for a person found in JSON. Should not happen. Details: {}".format( oc_person_uuid_to_json_map )
            print( status_message )
            
        else:
            
            # neither 1 or > 1, but greater than 0. Error.
            status_message = "- !!!! ERROR - neither 1 or > 1, but greater than 0. Impossible. Details: {}".format( oc_person_uuid_to_json_map )
            print( status_message )
            
        #-- END check for unexpected outcome --#
                
    else:
        
        # no person found in JSON - big error, not sure what we'll do.
        status_message = "- !!!! ERROR: Person NOT FOUND in JSON ( Person: {my_person}; Article_Data: {article_data}.".format(
            my_person = my_person,
            article_data = my_article_data
        )
        print( status_message )
        
    #-- END check to see if found person in JSON. --#
    
    # get IDs.
    no_name_id = no_name_instance.id
    article_data = no_name_instance.article_data
    article_data_id = article_data.id
    
    # output links
    print( "- Links:" )
    print( "" )
    print( "    - Article_Author {no_name_id} ( https://research.local/research/admin/context_text/article_author/{no_name_id}/change/ )".format( no_name_id = no_name_id ) )
    print( "    - Article_Data {AD_id} ( https://research.local/research/admin/context_text/article_data/{AD_id}/change/ )".format( AD_id = article_data_id ) )
    print( "    - Person {my_person_id} ( https://research.local/research/admin/context_text/person/{my_person_id}/change/ )".format( my_person_id = my_person_id ) )
    print( "    - Article {my_article_id} ( https://research.local/research/admin/context_text/article/{my_article_id}/change/ )".format( my_article_id = my_article_id ) )
    
#-- END loop over names QuerySet --#
